<a href="https://colab.research.google.com/github/abdulmajee/news_data_augmetation/blob/main/Fine_tuning_BERT_(dbmdz_bert_base_turkish_cased-augmented-data)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [ ]:
!pip install -q transformers datasets

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datasets
from datasets import load_dataset, DatasetDict


# dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Colab Notebooks/augmented_metin_dosyasi_dataframe.csv', split = 'train')

Using custom data configuration default-3a8fbb40032fb4a9
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-3a8fbb40032fb4a9/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [ ]:
dataset

Dataset({
    features: ['text', 'bilgilHealthy_Living', 'Technology_&_Computing', 'Music_and_Audio', 'Fine_Art', 'Pop_Culture', 'Family_and_Relationships', 'Science', 'Education', 'Hobbies_&_Interests', 'Movies', 'Food_&_Drink', 'Business_and_Finance', 'Healthy_Living', 'Medical_Health', 'Television', 'Personal_Finance', 'Style_&_Fashion', 'Real_Estate', 'Books_and_Literature', 'Pets', 'Sports', 'Travel', 'Shopping', 'Automotive', 'Careers', 'News_and_Politics', 'Home_&_Garden', 'Events_and_Attractions', 'Religion_&_Spirituality', 'Video_Gaming'],
    num_rows: 18623
})

In [ ]:
dataset = dataset.remove_columns("bilgilHealthy_Living")

In [ ]:
metin_train_devtest = dataset.train_test_split(shuffle = True, seed = 200, test_size=0.1)
metin_dev_test = metin_train_devtest['test'].train_test_split(shuffle = True, seed = 200, test_size=0.50)
dataset = DatasetDict({
    'train': metin_train_devtest['train'],
    'test': metin_dev_test['test'],
    'dev': metin_dev_test['train']})

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-3a8fbb40032fb4a9/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-e078966d5360e1ef.arrow and /root/.cache/huggingface/datasets/csv/default-3a8fbb40032fb4a9/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-8c664e24f0bed682.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-3a8fbb40032fb4a9/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-69fec0597154a657.arrow and /root/.cache/huggingface/datasets/csv/default-3a8fbb40032fb4a9/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-45d22c0d98b0af6a.arrow


Let's check the first example of the training split:

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'Technology_&_Computing', 'Music_and_Audio', 'Fine_Art', 'Pop_Culture', 'Family_and_Relationships', 'Science', 'Education', 'Hobbies_&_Interests', 'Movies', 'Food_&_Drink', 'Business_and_Finance', 'Healthy_Living', 'Medical_Health', 'Television', 'Personal_Finance', 'Style_&_Fashion', 'Real_Estate', 'Books_and_Literature', 'Pets', 'Sports', 'Travel', 'Shopping', 'Automotive', 'Careers', 'News_and_Politics', 'Home_&_Garden', 'Events_and_Attractions', 'Religion_&_Spirituality', 'Video_Gaming'],
        num_rows: 16760
    })
    test: Dataset({
        features: ['text', 'Technology_&_Computing', 'Music_and_Audio', 'Fine_Art', 'Pop_Culture', 'Family_and_Relationships', 'Science', 'Education', 'Hobbies_&_Interests', 'Movies', 'Food_&_Drink', 'Business_and_Finance', 'Healthy_Living', 'Medical_Health', 'Television', 'Personal_Finance', 'Style_&_Fashion', 'Real_Estate', 'Books_and_Literature', 'Pets', 'Sports', 'Travel', 'Shopping

In [ ]:
example = dataset['train'][0]
example

{'Automotive': 0,
 'Books_and_Literature': 0,
 'Business_and_Finance': 0,
 'Careers': 0,
 'Education': 0,
 'Events_and_Attractions': 0,
 'Family_and_Relationships': 0,
 'Fine_Art': 0,
 'Food_&_Drink': 0,
 'Healthy_Living': 0,
 'Hobbies_&_Interests': 0,
 'Home_&_Garden': 0,
 'Medical_Health': 0,
 'Movies': 0,
 'Music_and_Audio': 0,
 'News_and_Politics': 1,
 'Personal_Finance': 0,
 'Pets': 0,
 'Pop_Culture': 0,
 'Real_Estate': 1,
 'Religion_&_Spirituality': 0,
 'Science': 0,
 'Shopping': 0,
 'Sports': 0,
 'Style_&_Fashion': 0,
 'Technology_&_Computing': 0,
 'Television': 0,
 'Travel': 0,
 'Video_Gaming': 0,
 'text': 'yazlıklar fiyatları yüzde 40 azaldı son dakika ekonomi haberleri ntv haber capcanli i̇zle foto galeri̇ vi̇deo galeri̇ foto galeri̇ vi̇deo galeri̇ ntv programlari yayin akiş ntv 360 ntv webi̇nar mi̇lli̇ pi̇yango corona vi̇rüs son daki̇ka türki ekonomi̇ dünya sanat yaşam sağlik otomobi̇l seyahat eği̇ti̇m teknoloji̇ haberleri ekonomi haberleri yazlıklar fiyatları yüzde 40 azald

The dataset consists of text documents, labeled with one or more categories. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['Technology_&_Computing',
 'Music_and_Audio',
 'Fine_Art',
 'Pop_Culture',
 'Family_and_Relationships',
 'Science',
 'Education',
 'Hobbies_&_Interests',
 'Movies',
 'Food_&_Drink',
 'Business_and_Finance',
 'Healthy_Living',
 'Medical_Health',
 'Television',
 'Personal_Finance',
 'Style_&_Fashion',
 'Real_Estate',
 'Books_and_Literature',
 'Pets',
 'Sports',
 'Travel',
 'Shopping',
 'Automotive',
 'Careers',
 'News_and_Politics',
 'Home_&_Garden',
 'Events_and_Attractions',
 'Religion_&_Spirituality',
 'Video_Gaming']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Parameter 'function'=<function preprocess_data at 0x7f26e71a59e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] yazlıklar fiyatları yüzde 40 azaldı son dakika ekonomi haberleri ntv haber capcanli i̇zle foto galeri̇ vi̇deo galeri̇ foto galeri̇ vi̇deo galeri̇ ntv programlari yayin akiş ntv 360 ntv webi̇nar mi̇lli̇ pi̇yango corona vi̇rüs son daki̇ka türki ekonomi̇ dünya sanat yaşam sağlik otomobi̇l seyahat eği̇ti̇m teknoloji̇ haberleri ekonomi haberleri yazlıklar fiyatları yüzde 40 azaldı yazlıklar fiyatları yüzde 40 azaldı pandemiyle öne çıkan yazlıklar evlerde fiyatlar geçen [SEP]'

In [ ]:
example['labels']

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Real_Estate', 'News_and_Politics']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [ ]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-base-turkish-cased-finetuned-metin-dosya-turkish",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([    2, 23774,  1980,  5666,  2902,  4062, 23557,  2094,  3774,  8449,
         9000,    82, 23326,  3013, 25964,  3709,  2031,    77,  1305,  1050,
         1985, 17772, 26878,  1305,    90,  1024,  1305,  1988,  1033, 26878,
         1305, 17772, 26878,  1305,    90,  1024,  1305,  1988,  1033, 26878,
         1305,    82, 23326, 14197,  1024,  2617,  1973,  2332,  2020,    82,
        23326, 17233,    82, 23326,  4528,  1024,  1305, 25747,  2546,  1305,
        18483,  1024,  1305, 11551,  1305,  2414,  4634,  9751,  5310,  1006,
           90,  1024,  1305,  9025,  1022,  2094,  4317,  1305,  2163,  5439,
         1024,  8449,  1305,  3078,  3517,  3315,  2267,  2102,  5146, 14372,
         3521,  1305,  1008,  6312,  3616,  1024,  1305,  2063,  1305,  1027,
         6228,  1305,  9000,  8449,  9000, 23774,  1980,  5666,  2902,  4062,
        23557, 23774,  1980,  5666,  2902,  4062, 23557,  4862,  2788,  2260,
         5342,  3671, 23774,  1980, 21647, 11964,  3227,     3])

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput([('loss',
                           tensor(0.7185, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)),
                          ('logits',
                           tensor([[-0.1584,  0.1162,  0.3475, -0.3454,  0.9387, -0.5257, -0.3864, -0.5752,
                                     0.0440,  0.2185, -0.3111,  0.2681, -0.0435,  0.1962,  0.0878,  1.0833,
                                    -0.3022, -0.9510, -0.5891, -0.3538, -0.3740, -0.1904,  0.4151, -0.1612,
                                     0.6260,  0.5708,  0.6701, -0.0612,  0.2883]],
                                  grad_fn=<AddmmBackward0>))])

Let's start training!

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16760
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10475


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.040000,0.028174,0.948401,0.956907,0.883996
2,0.015600,0.010802,0.989622,0.990112,0.974221
3,0.007500,0.005350,0.996705,0.997494,0.992481
4,0.004300,0.003296,0.998767,0.999550,0.997852
5,0.003300,0.002635,1.000000,1.000000,1.000000


***** Running Evaluation *****
  Num examples = 931
  Batch size = 8
Saving model checkpoint to bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-2095
Configuration saved in bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-2095/config.json
Model weights saved in bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-2095/pytorch_model.bin
tokenizer config file saved in bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-2095/tokenizer_config.json
Special tokens file saved in bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-2095/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 931
  Batch size = 8
Saving model checkpoint to bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-4190
Configuration saved in bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-4190/config.json
Model weights saved in bert-base-turkish-cased-finetuned-metin-dosya-turkish/checkpoint-4190/pytorch_

TrainOutput(global_step=10475, training_loss=0.026168471691432032, metrics={'train_runtime': 2473.5274, 'train_samples_per_second': 33.879, 'train_steps_per_second': 4.235, 'total_flos': 5513512883251200.0, 'train_loss': 0.026168471691432032, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_loss': 0.002634634030982852,
 'eval_roc_auc': 1.0,
 'eval_runtime': 8.0892,
 'eval_samples_per_second': 115.092,
 'eval_steps_per_second': 14.464}

## Inference

Let's test the model on a new sentence:

In [ ]:
text = "Koca, Twitter hesabından yaptığı paylaşımda,Şoför arkadaşımız arzusu üzerine misafirim oldu. Hasta nakil aracıyla naklini yaptığı hastanın Ankara Şehir Hastanesi aciline yatışını sağladıktan 2,5 saat sonra aracıyla bakanlık binası önüne park edişinden başladık, çok şey konuştuk. Ama bazı şeyler konuşulduğu yerde kalmalı"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 29])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['News_and_Politics']
